In [1]:
import pandas as pd

In [2]:
# metro
metro_df = pd.read_csv(
    "./../data/preproc/metro/metro_v1_final.csv", index_col=0
)

In [3]:
metro_df["날짜"] = pd.to_datetime(metro_df["날짜"])

In [4]:
# metro
station_location_df = pd.read_csv(
    "./../data/asset/서울교통공사 역주소 및 전화번호_20230831.csv",
    encoding="cp949",
)

In [5]:
class CustomDict(dict):
    def __missing__(self,key):
        return key

_rename_dict = CustomDict()
_rename_dict["서울"] = "서울역"

In [6]:
station_location_df['역명'] = station_location_df['역명'].apply(lambda x : _rename_dict[x])
station_location_df['지번주소'] = station_location_df['지번주소'].apply(lambda x : '_'.join(x.split()[:3]))

station_location_dict = station_location_df.set_index("역명")['지번주소'].to_dict()

In [7]:
metro_df.index = metro_df.index.map(station_location_dict)
metro_df = metro_df.reset_index()
metro_df['year'] = metro_df['날짜'].apply(lambda x : x.year)

In [8]:
metro_df = (
    metro_df.dropna().sort_values("total_out").groupby(["역명", "year"]).tail(1)
)

In [9]:
metro_df.columns = ["location", "date", "score", "year"]

In [10]:
pivot_df = (
    metro_df.reset_index()
    .pivot(index="location", values="score", columns="year")
    .fillna(0)
)

In [11]:
metro_df = pivot_df

In [12]:
metro_df

year,2017,2018,2019,2020,2021,2022
location,,,,,,
경기도_고양시_덕양구,1.393408,1.310458,1.470133,1.401607,1.359967,1.522072
경기도_광명시_광명동,1.192534,1.216496,1.207054,1.509130,1.244506,1.309381
경기도_광명시_철산동,1.289224,1.288180,1.315625,1.550433,1.318775,1.368058
경기도_성남시_수정구,1.678033,1.783348,1.974753,1.841216,1.703500,1.689261
경기도_의정부시_장암동,1.856264,1.847157,1.713237,1.705467,1.506951,1.614401
...,...,...,...,...,...,...
서울특별시_중랑구_면목동,1.386639,1.281403,1.282092,1.376933,1.293595,1.352534
서울특별시_중랑구_묵동,1.347463,1.272394,1.351765,1.452952,1.251996,1.306771
서울특별시_중랑구_상봉동,1.230678,1.246334,1.261111,1.463706,1.275123,1.296504


In [13]:
metro_df.to_csv("./metro.csv")